Creating a Dataframe

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf

spark = SparkSession.builder.appName("Netflix_Title").getOrCreate()

22/10/20 13:31:29 WARN Utils: Your hostname, Inspiron-5567 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlp2s0)
22/10/20 13:31:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 13:31:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/20 13:31:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/20 13:31:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df = spark.read.json('/home/saurav/Desktop/SparkAssignment/netflix_titles.json')

In [3]:
df.select(df['show_id']).show()

+-------+
|show_id|
+-------+
|     s1|
|     s2|
|     s3|
|     s4|
|     s5|
|     s6|
|     s7|
|     s8|
|     s9|
|    s10|
|    s11|
|    s12|
|    s13|
|    s14|
|    s15|
|    s16|
|    s17|
|    s18|
|    s19|
|    s20|
+-------+
only showing top 20 rows



# 1. How many PG-13 titles are there?

In [4]:
pg13_count = df.where(df.rating=='PG-13').count()
print("Total PG-13 Titles:",pg13_count)

Total PG-13 Titles: 386


# 2. How many titles an actor or actress appeared in?

In [5]:
split_cast=df.select(df.cast).withColumn('cast',F.explode(F.split('cast',',')))

cast_count= split_cast.groupBy('cast').count().orderBy('count', ascending=False)# groupby 'cast' and count the number of times each ac

remove_empty_cast = cast_count.filter(cast_count.cast!='')

total_cast_app = remove_empty_cast.show()

+------------------+-----+
|              cast|count|
+------------------+-----+
|       Anupam Kher|   37|
|  Takahiro Sakurai|   27|
|           Om Puri|   27|
|    Shah Rukh Khan|   27|
|       Boman Irani|   25|
|      Paresh Rawal|   24|
|     Andrea Libman|   24|
|         Yuki Kaji|   23|
|      Akshay Kumar|   22|
|      Vincent Tong|   22|
|       John Cleese|   20|
|    Kareena Kapoor|   20|
|      Adam Sandler|   19|
|    Junichi Suwabe|   18|
|         Ai Kayano|   18|
|  Amitabh Bachchan|   18|
|    Gulshan Grover|   18|
|David Attenborough|   18|
|  Naseeruddin Shah|   18|
|    Yashpal Sharma|   18|
+------------------+-----+
only showing top 20 rows



# 3. How many titles has a director has filmed?


In [6]:
split_director = df.select(df.director).withColumn('director',F.explode(F.split('director',',')))

count_director = split_director.groupBy('director').count().orderBy('director',ascending=False)

remove_empty_count = count_director.filter(count_director.director !='')

total_director = remove_empty_count.show()

# df.groupBy("director").count().show()

+--------------------+-----+
|            director|count|
+--------------------+-----+
|        Şenol Sönmez|    2|
|    Ömer Faruk Sorak|    2|
| Óskar Thór Axelsson|    1|
|    Ísold Uggadóttir|    1|
|         Çagan Irmak|    1|
|Ángel Gómez Herná...|    1|
|     Álvaro Longoria|    1|
|Álvaro Delgado-Ap...|    1|
|     Álvaro Brechner|    1|
|  Álex de la Iglesia|    2|
|         Àlex Pastor|    2|
|         Zuko Nodada|    1|
|         Zsolt Pálfi|    1|
|         Zoya Akhtar|    4|
|    Zoe Lister-Jones|    1|
|       Zoe Berriatúa|    2|
|       Ziggy Hofmeyr|    1|
|           Ziga Virc|    1|
|        Ziad Doueiri|    2|
|         Zhang Yimou|    1|
+--------------------+-----+
only showing top 20 rows



# 4. What content is available in different countries?

In [7]:
df.groupBy('country','listed_in').count().drop('count').show()


+--------------------+--------------------+
|             country|           listed_in|
+--------------------+--------------------+
|              Turkey|Dramas, Internati...|
|            Colombia|Documentaries, In...|
|           Australia|Dramas, Internati...|
|               Egypt|International TV ...|
|                    |Crime TV Shows, I...|
|            Pakistan|Dramas, Internati...|
|Mexico, United St...|            Comedies|
|             Germany|Comedies, Interna...|
|              Mexico|Action & Adventur...|
|United Kingdom, U...|Action & Adventur...|
|Canada, United St...|Horror Movies, Th...|
|               Japan|International TV ...|
|       United States|Action & Adventur...|
|               Italy|Comedies, Interna...|
|       United States|Comedies, Indepen...|
|         South Korea|Action & Adventur...|
|               Chile|Comedies, Indepen...|
|           Argentina|Kids' TV, Spanish...|
|                    |Crime TV Shows, I...|
|   Denmark, Zimbabwe|Documentar

# 5. How many movies were released in 2008?

In [8]:
total_2008 = df.where((df.release_year==2008) &  (df.type  == "Movie")).count()
print("Total movies released in 2008 is:",total_2008)

Total movies released in 2008 is: 102


# 6.List all the movies whose duration is greater than 100 mins ?

In [9]:
# dur= df.where(df.type=="Movie").select("duration").show()
def duration_100(replace_min):
    return replace_min.replace('min','')

durationUDF = udf(duration_100)

filter_movie = df.filter(df['type']=='Movie').select(df['title'],df['duration'])

replacing_min = filter_movie.select(df['title'],durationUDF(filter_movie['duration']))
duration_100 =  replacing_min.where(replacing_min['duration_100(duration)']>100).show()


+--------------------+----------------------+
|               title|duration_100(duration)|
+--------------------+----------------------+
|                  21|                  123 |
|                 187|                  119 |
|                 706|                  118 |
|                1920|                  143 |
|                1922|                  103 |
|              Oct-01|                  149 |
|              22-Jul|                  144 |
|              15-Aug|                  124 |
|​​Kuch Bheege Alfaaz|                  110 |
|        ​Goli Soda 2|                  128 |
|  ​Maj Rati ​​Keteki|                  117 |
|   #FriendButMarried|                  102 |
| #FriendButMarried 2|                  104 |
|               #Roxy|                  105 |
|             #Selfie|                  125 |
|          #Selfie 69|                  119 |
|          Ég man þig|                  106 |
|             Òlòtūré|                  107 |
|       1 Mile to You|            

# 7.List movies played by “Kareena Kapoor” ?

In [10]:
split_cast=df.select(df.cast,df.title).withColumn('cast',F.explode(F.split('cast',',')))
kareena = split_cast.filter(split_cast['cast']=='Kareena Kapoor').show()
# cast_count= split_cast.groupBy('cast').count().orderBy('count', ascending=False)
# remove_empty_cast = cast_count.filter((cast_count.cast!='') & (cast_count.cast=='Kareena Kapoor')).show()


+--------------+------------------+
|          cast|             title|
+--------------+------------------+
|Kareena Kapoor|           Chameli|
|Kareena Kapoor|Ek Main Aur Ekk Tu|
|Kareena Kapoor|           Heroine|
|Kareena Kapoor|           Kurbaan|
|Kareena Kapoor|        Satyagraha|
+--------------+------------------+

